In [1]:
import pandas as pd
import numpy as np

# --- PASO 1: Cargar los datos de prueba en DataFrames ---
# 

# Creación de customers_raw
customers_data = {
    'customer_id': [1, 2, 3, 4, 5],
    'name': ['Ana Torres', 'Juan Pérez', 'Laura Gómez', 'Juan Pérez', np.nan], # np.nan es el NULL
    'email': ['ana@email.com', np.nan, 'laura_gomez@email.com', 'juanperez@email.com', 'andres@email.com'], # np.nan es el NULL
    'country': ['Colombia', 'Mexico', np.nan, 'Mexico', 'Chile'] # np.nan es el NULL
}
customers_raw_df = pd.DataFrame(customers_data)

# Creación de transactions_raw
transactions_data = {
    'transaction_id': [100, 101, 102, 103, 104],
    'customer_id': [1, 2, 2, 3, 6], # El '6' es el huérfano
    'amount': [200.0, 150.0, 150.0, np.nan, 300.0], # np.nan es el NULL
    'date': pd.to_datetime(['2025-01-01', '2025-01-02', '2025-01-02', '2025-01-03', '2025-01-04'])
}
transactions_raw_df = pd.DataFrame(transactions_data)

# Creación de transactions_clean (CON ERRORES SIMULADOS)
transactions_clean_data = {
    'transaction_id': [100, 101, 102, 105],
    'amount': [200.0, 150.0, 140.0, 50.0], # Error: 140.0 en lugar de 150.0
    'country': ['Colombia', 'Mexico', 'Mexico', 'Argentina'] # Error: Argentina es fantasma
}
transactions_clean_df = pd.DataFrame(transactions_clean_data)


print("--- DataFrames de Origen Cargados ---")
print("\n[ customers_raw_df ]")
print(customers_raw_df)
print("\n[ transactions_raw_df ]")
print(transactions_raw_df)
print("\n[ transactions_clean_df (Simulada con errores) ]")
print(transactions_clean_df)
print("-" * 40)

--- DataFrames de Origen Cargados ---

[ customers_raw_df ]
   customer_id         name                  email   country
0            1   Ana Torres          ana@email.com  Colombia
1            2   Juan Pérez                    NaN    Mexico
2            3  Laura Gómez  laura_gomez@email.com       NaN
3            4   Juan Pérez    juanperez@email.com    Mexico
4            5          NaN       andres@email.com     Chile

[ transactions_raw_df ]
   transaction_id  customer_id  amount       date
0             100            1   200.0 2025-01-01
1             101            2   150.0 2025-01-02
2             102            2   150.0 2025-01-02
3             103            3     NaN 2025-01-03
4             104            6   300.0 2025-01-04

[ transactions_clean_df (Simulada con errores) ]
   transaction_id  amount    country
0             100   200.0   Colombia
1             101   150.0     Mexico
2             102   140.0     Mexico
3             105    50.0  Argentina
--------------

In [3]:
print("\n--- INICIO PRUEBA 1: COMPLETITUD ---")

# Definir los campos obligatorios para cada tabla
campos_obligatorios_cust = ['name', 'email', 'country']
campos_obligatorios_trans = ['amount']

# 1. Validar customers_raw
# .isnull() crea un DF de Boleanos, .sum() los cuenta por columna
errores_completitud_cust = customers_raw_df[campos_obligatorios_cust].isnull().sum()

# Filtramos para mostrar solo las columnas que SÍ tienen errores
errores_completitud_cust = errores_completitud_cust[errores_completitud_cust > 0]

if not errores_completitud_cust.empty:
    print("🔴 FALLIDO: Se encontraron NULOS en 'customers_raw':")
    print(errores_completitud_cust)
else:
    print("🟢 APROBADO: 'customers_raw' está completa.")

# 2. Validar transactions_raw
errores_completitud_trans = transactions_raw_df[campos_obligatorios_trans].isnull().sum()
errores_completitud_trans = errores_completitud_trans[errores_completitud_trans > 0]

if not errores_completitud_trans.empty:
    print("🔴 FALLIDO: Se encontraron NULOS en 'transactions_raw':")
    print(errores_completitud_trans)
else:
    print("🟢 APROBADO: 'transactions_raw' está completa.")


--- INICIO PRUEBA 1: COMPLETITUD ---
🔴 FALLIDO: Se encontraron NULOS en 'customers_raw':
name       1
email      1
country    1
dtype: int64
🔴 FALLIDO: Se encontraron NULOS en 'transactions_raw':
amount    1
dtype: int64


In [4]:
print("\n--- INICIO PRUEBA 2: UNICIDAD (EMAIL) ---")

# 1. Replicamos 'WHERE email IS NOT NULL'
df_sin_nulos = customers_raw_df.dropna(subset=['email'])

# 2. Replicamos 'GROUP BY / HAVING COUNT > 1'
# .duplicated() busca duplicados. 'keep=False' marca todas las ocurrencias.
filas_duplicadas = df_sin_nulos[df_sin_nulos.duplicated(subset=['email'], keep=False)]

if not filas_duplicadas.empty:
    print(f"🔴 FALLIDO: Se encontraron {len(filas_duplicadas)} registros con email duplicado:")
    print(filas_duplicadas.sort_values(by='email'))
else:
    print("🟢 APROBADO: No se encontraron emails duplicados.")


--- INICIO PRUEBA 2: UNICIDAD (EMAIL) ---
🟢 APROBADO: No se encontraron emails duplicados.


In [5]:
print("\n--- INICIO PRUEBA 3: INTEGRIDAD REFERENCIAL ---")

# 1. Hacemos el LEFT JOIN (how='left') desde transacciones hacia clientes
#    indicator=True es el "asistente" que nos dice qué falló
merged_df = pd.merge(
    transactions_raw_df,
    customers_raw_df,
    on='customer_id',
    how='left',
    indicator=True # ¡La magia! Crea una columna '_merge'
)

# 2. Filtramos donde el indicador sea 'left_only' (equivalente a 'WHERE c.customer_id IS NULL')
registros_huerfanos = merged_df[merged_df['_merge'] == 'left_only']

if not registros_huerfanos.empty:
    print(f"🔴 FALLIDO: Se encontraron {len(registros_huerfanos)} transacciones huérfanas:")
    # Mostramos solo las columnas relevantes del reporte de error
    # Nota: Corregí 'customer_id_trans' a 'customer_id' para que el script se pueda ejecutar
    # ya que el DataFrame original se llama 'customer_id'.
    print(registros_huerfanos[['transaction_id', 'customer_id']])
else:
    print("🟢 APROBADO: No se encontraron transacciones huérfanas.")


--- INICIO PRUEBA 3: INTEGRIDAD REFERENCIAL ---
🔴 FALLIDO: Se encontraron 1 transacciones huérfanas:
   transaction_id  customer_id
4             104            6


In [6]:
print("\n--- INICIO PRUEBA 4: RECONCILIACIÓN ---")

# --- Fase 1: Calcular Totales de Origen (limpios) ---
# 1. Unir (INNER JOIN por defecto)
origen_merged = pd.merge(transactions_raw_df, customers_raw_df, on='customer_id')

# 2. Filtrar (WHERE ... IS NOT NULL)
origen_filtrado = origen_merged.dropna(subset=['amount', 'country'])

# 3. Agrupar (GROUP BY / SUM)
totales_origen = origen_filtrado.groupby('country')['amount'].sum().reset_index()
totales_origen = totales_origen.rename(columns={'amount': 'total_origen'})

# --- Fase 2: Calcular Totales de Destino ---
totales_destino = transactions_clean_df.groupby('country')['amount'].sum().reset_index()
totales_destino = totales_destino.rename(columns={'amount': 'total_destino'})

# --- Fase 3: Comparar (FULL OUTER JOIN) ---
# 1. how='outer' es el FULL OUTER JOIN
reporte_final = pd.merge(
    totales_origen,
    totales_destino,
    on='country',
    how='outer'
)

# 2. Rellenar NULLs con 0 (el COALESCE)
reporte_final = reporte_final.fillna(0)

# 3. Calcular la diferencia
reporte_final['diferencia'] = reporte_final['total_origen'] - reporte_final['total_destino']

# 4. Filtrar (WHERE diferencia != 0)
errores_reconciliacion = reporte_final[reporte_final['diferencia'] != 0]

if not errores_reconciliacion.empty:
    print(f"🔴 FALLIDO: Se encontraron {len(errores_reconciliacion)} discrepancias en la reconciliación:")
    print(errores_reconciliacion)
else:
    print("🟢 APROBADO: La reconciliación de totales es exitosa.")


--- INICIO PRUEBA 4: RECONCILIACIÓN ---
🔴 FALLIDO: Se encontraron 2 discrepancias en la reconciliación:
     country  total_origen  total_destino  diferencia
0  Argentina           0.0           50.0       -50.0
2     Mexico         300.0          290.0        10.0
